In [ ]:
# 必要なライブラリをインポート
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
import logging
from tqdm import tqdm
import matplotlib.pyplot as plt
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
from sqlalchemy import create_engine, text
import torch.nn.functional as F
import japanize_matplotlib

# 自作モジュールをインポート
from _dataset import HorguesDataset
from _models import HorguesModel
from horgues3.losses import HorguesLoss
from horgues3.odds import get_odds_dataframes
from horgues3.haraimodoshi import get_haraimodoshi_dataframes
from horgues3.database import create_database_engine

In [ ]:
# ログ設定
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [ ]:
# デバイス設定
device = torch.device('cuda:1')
logger.info(f"使用デバイス: {device}")

In [ ]:
# 学習パラメータ設定
BATCH_SIZE = 128

In [ ]:
checkpoint = torch.load('outputs/best_model.pth', map_location='cpu')

In [ ]:
# 学習期間・検証期間設定
START_DATE = '20230101'
END_DATE = '20231231'

logger.info(f"期間: {START_DATE} - {END_DATE}")

In [ ]:
# データセットの作成
logger.info("データセットを作成中...")
dataset = HorguesDataset(
    start_date=START_DATE,
    end_date=END_DATE,
    preprocessing_params=checkpoint['preprocessing_params'],
    cache_dir='cache/test',
    use_cache=True
)

logger.info(f"データサイズ: {len(dataset)}")

In [ ]:
loader = DataLoader(
    dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

In [ ]:
# モデル設定の取得
model_config = checkpoint['model_config']
logger.info(f"数値特徴量数: {len(model_config['numerical_features'])}")
logger.info(f"カテゴリ特徴量数: {len(model_config['categorical_features'])}")
logger.info(f"時系列データ: {model_config['sequence_names']}")

In [ ]:
# モデルの作成
model = HorguesModel(
    sequence_names=model_config['sequence_names'],
    feature_aliases=model_config['feature_aliases'],
    numerical_features=model_config['numerical_features'],
    categorical_features=model_config['categorical_features'],
).to(device)

In [ ]:
model.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
race_ids = []
masks = []
targets = []
scores = []

model.eval()
with torch.no_grad():
    pbar = tqdm(loader, desc='Predict')
    for batch in pbar:
        race_ids.extend(batch['race_id'])
        masks.append(batch['mask'])
        targets.append(batch['target'])

        x_num = {k: v.to(device) for k, v in batch['x_num'].items()}
        x_cat = {k: v.to(device) for k, v in batch['x_cat'].items()}
        sequence_data = {}
        for seq_name, seq_data in batch['sequence_data'].items():
            sequence_data[seq_name] = {
                'x_num': {k: v.to(device) for k, v in seq_data['x_num'].items()},
                'x_cat': {k: v.to(device) for k, v in seq_data['x_cat'].items()},
                'mask': seq_data['mask'].to(device)
            }
        mask = batch['mask'].to(device)
        target = batch['target'].to(device)

        scores.append(model(x_num, x_cat, sequence_data, mask).cpu())

    masks = torch.cat(masks, dim=0)
    targets = torch.cat(targets, dim=0)
    scores = torch.cat(scores, dim=0)

In [ ]:
odds = get_odds_dataframes(START_DATE, END_DATE)
haraimodoshi = get_haraimodoshi_dataframes(START_DATE, END_DATE)

In [ ]:
def convert_to_series(tensor, name):
    tmp = pd.DataFrame(tensor, index=race_ids, columns=np.arange(1, 19))
    tmp = tmp.reset_index(names='race_id').melt(id_vars='race_id', var_name='uma_number')
    tmp = tmp.set_index(['race_id', 'uma_number']).value.rename(name)
    return tmp

In [ ]:
df = pd.DataFrame(
    {
        'score_0': convert_to_series(F.sigmoid(scores[:, :, 0]), 'score_0'),
        'score_1': convert_to_series(F.sigmoid(scores[:, :, 1]), 'score_1'),
        'score_2': convert_to_series(F.sigmoid(scores[:, :, 2]), 'score_2'),
        'target_0': convert_to_series(targets[:, :, 0], 'target_0'),
        'target_1': convert_to_series(targets[:, :, 1], 'target_1'),
        'target_2': convert_to_series(targets[:, :, 2], 'target_2'),
        'mask': convert_to_series(masks, 'mask'),
    }
)
df = df[df['mask']].sort_index()

In [ ]:
df['score_ranking_0'] = df.groupby('race_id')['score_0'].rank(method='first', ascending=False)
df['score_ranking_1'] = df.groupby('race_id')['score_1'].rank(method='first', ascending=False)
df['score_ranking_2'] = df.groupby('race_id')['score_2'].rank(method='first', ascending=False)

In [ ]:
# 全馬券種購入シミュレーション
from itertools import product, combinations, permutations
import pandas as pd
import numpy as np
from concurrent.futures import ProcessPoolExecutor, as_completed
from functools import partial
import multiprocessing as mp
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
import seaborn as sns
import japanize_matplotlib

def simulate_tansho_betting(df, odds, haraimodoshi, thresholds):
    """
    単勝購入シミュレーション
    """
    results = []
    
    for race_id in df.index.get_level_values('race_id').unique():
        race_df = df.loc[race_id]
        
        # 1着候補の選定
        candidates = race_df[
            (race_df['score_0'] >= thresholds['score_0']) & 
            (race_df['score_ranking_0'] <= thresholds['score_ranking_0'])
        ].index.values
        
        total_bet = 0
        payout = 0
        hit_count = 0
        
        for horse in candidates:
            horse_str = f"{horse:02d}"
            
            # オッズをチェック
            if race_id in odds['tansho'].index and horse_str in odds['tansho'].columns:
                odds_value = odds['tansho'].loc[race_id, horse_str]
                if odds_value >= thresholds['min_odds']:
                    total_bet += 100
                    
                    # 的中チェック
                    if race_id in haraimodoshi['tansho'].index:
                        payout_value = haraimodoshi['tansho'].loc[race_id, horse_str]
                        if payout_value > 0:
                            payout += payout_value
                            hit_count += 1
        
        if total_bet > 0:
            results.append({
                'race_id': race_id,
                'total_bet': total_bet,
                'payout': payout,
                'hit_count': hit_count,
                'purchase_count': len(candidates)
            })
    
    return pd.DataFrame(results)

def simulate_fukusho_betting(df, odds, haraimodoshi, thresholds):
    """
    複勝購入シミュレーション
    """
    results = []
    
    for race_id in df.index.get_level_values('race_id').unique():
        race_df = df.loc[race_id]
        
        # 3着以内候補の選定（複勝は1-3着全ての予測を考慮）
        candidates = race_df[
            ((race_df['score_0'] >= thresholds['score_0']) & (race_df['score_ranking_0'] <= thresholds['score_ranking_0'])) |
            ((race_df['score_1'] >= thresholds['score_1']) & (race_df['score_ranking_1'] <= thresholds['score_ranking_1'])) |
            ((race_df['score_2'] >= thresholds['score_2']) & (race_df['score_ranking_2'] <= thresholds['score_ranking_2']))
        ].index.values
        
        total_bet = 0
        payout = 0
        hit_count = 0
        
        for horse in candidates:
            horse_str = f"{horse:02d}"
            
            # オッズをチェック
            if race_id in odds['fukusho'].index and horse_str in odds['fukusho'].columns:
                odds_value = odds['fukusho'].loc[race_id, horse_str]
                if odds_value >= thresholds['min_odds']:
                    total_bet += 100
                    
                    # 的中チェック
                    if race_id in haraimodoshi['fukusho'].index:
                        payout_value = haraimodoshi['fukusho'].loc[race_id, horse_str]
                        if payout_value > 0:
                            payout += payout_value
                            hit_count += 1
        
        if total_bet > 0:
            results.append({
                'race_id': race_id,
                'total_bet': total_bet,
                'payout': payout,
                'hit_count': hit_count,
                'purchase_count': len(candidates)
            })
    
    return pd.DataFrame(results)

def simulate_umaren_betting(df, odds, haraimodoshi, thresholds):
    """
    馬連購入シミュレーション（BOX買い）
    """
    results = []
    
    for race_id in df.index.get_level_values('race_id').unique():
        race_df = df.loc[race_id]
        
        # 1着候補と2着候補の選定
        first_candidates = race_df[
            (race_df['score_0'] >= thresholds['score_0']) & 
            (race_df['score_ranking_0'] <= thresholds['score_ranking_0'])
        ].index.values
        
        second_candidates = race_df[
            (race_df['score_1'] >= thresholds['score_1']) & 
            (race_df['score_ranking_1'] <= thresholds['score_ranking_1'])
        ].index.values
        
        # BOX買いの候補（1着または2着候補）
        all_candidates = list(set(list(first_candidates) + list(second_candidates)))
        
        total_bet = 0
        payout = 0
        hit_count = 0
        
        # 2頭BOXの組み合わせ
        for horse1, horse2 in combinations(all_candidates, 2):
            combination = f"{min(horse1, horse2):02d}-{max(horse1, horse2):02d}"
            
            # オッズをチェック
            if race_id in odds['umaren'].index and combination in odds['umaren'].columns:
                odds_value = odds['umaren'].loc[race_id, combination]
                if odds_value >= thresholds['min_odds']:
                    total_bet += 100
                    
                    # 的中チェック
                    if race_id in haraimodoshi['umaren'].index:
                        payout_value = haraimodoshi['umaren'].loc[race_id, combination]
                        if payout_value > 0:
                            payout = payout_value
                            hit_count = 1
                            break
        
        if total_bet > 0:
            results.append({
                'race_id': race_id,
                'total_bet': total_bet,
                'payout': payout,
                'hit_count': hit_count,
                'purchase_count': len(list(combinations(all_candidates, 2)))
            })
    
    return pd.DataFrame(results)

def simulate_wide_betting(df, odds, haraimodoshi, thresholds):
    """
    ワイド購入シミュレーション（BOX買い）
    """
    results = []
    
    for race_id in df.index.get_level_values('race_id').unique():
        race_df = df.loc[race_id]
        
        # 3着以内候補の選定
        candidates = race_df[
            ((race_df['score_0'] >= thresholds['score_0']) & (race_df['score_ranking_0'] <= thresholds['score_ranking_0'])) |
            ((race_df['score_1'] >= thresholds['score_1']) & (race_df['score_ranking_1'] <= thresholds['score_ranking_1'])) |
            ((race_df['score_2'] >= thresholds['score_2']) & (race_df['score_ranking_2'] <= thresholds['score_ranking_2']))
        ].index.values
        
        total_bet = 0
        payout = 0
        hit_count = 0
        
        # 2頭BOXの組み合わせ
        for horse1, horse2 in combinations(candidates, 2):
            combination = f"{min(horse1, horse2):02d}-{max(horse1, horse2):02d}"
            
            # オッズをチェック
            if race_id in odds['wide'].index and combination in odds['wide'].columns:
                odds_value = odds['wide'].loc[race_id, combination]
                if odds_value >= thresholds['min_odds']:
                    total_bet += 100
                    
                    # 的中チェック
                    if race_id in haraimodoshi['wide'].index:
                        payout_value = haraimodoshi['wide'].loc[race_id, combination]
                        if payout_value > 0:
                            payout += payout_value  # ワイドは複数的中の可能性があるので加算
                            hit_count += 1
        
        if total_bet > 0:
            results.append({
                'race_id': race_id,
                'total_bet': total_bet,
                'payout': payout,
                'hit_count': hit_count,
                'purchase_count': len(list(combinations(candidates, 2)))
            })
    
    return pd.DataFrame(results)

def simulate_umatan_betting(df, odds, haraimodoshi, thresholds):
    """
    馬単購入シミュレーション（フォーメーション）
    """
    results = []
    
    for race_id in df.index.get_level_values('race_id').unique():
        race_df = df.loc[race_id]
        
        # 1着候補と2着候補の選定
        first_candidates = race_df[
            (race_df['score_0'] >= thresholds['score_0']) & 
            (race_df['score_ranking_0'] <= thresholds['score_ranking_0'])
        ].index.values
        
        second_candidates = race_df[
            (race_df['score_1'] >= thresholds['score_1']) & 
            (race_df['score_ranking_1'] <= thresholds['score_ranking_1'])
        ].index.values
        
        total_bet = 0
        payout = 0
        hit_count = 0
        
        # フォーメーション購入
        for first, second in product(first_candidates, second_candidates):
            if first != second:
                combination = f"{first:02d}-{second:02d}"
                
                # オッズをチェック
                if race_id in odds['umatan'].index and combination in odds['umatan'].columns:
                    odds_value = odds['umatan'].loc[race_id, combination]
                    if odds_value >= thresholds['min_odds']:
                        total_bet += 100
                        
                        # 的中チェック
                        if race_id in haraimodoshi['umatan'].index:
                            payout_value = haraimodoshi['umatan'].loc[race_id, combination]
                            if payout_value > 0:
                                payout = payout_value
                                hit_count = 1
                                break
        
        if total_bet > 0:
            results.append({
                'race_id': race_id,
                'total_bet': total_bet,
                'payout': payout,
                'hit_count': hit_count,
                'purchase_count': len([1 for first, second in product(first_candidates, second_candidates) if first != second])
            })
    
    return pd.DataFrame(results)

def simulate_sanrenpuku_betting(df, odds, haraimodoshi, thresholds):
    """
    3連複購入シミュレーション（BOX買い）
    """
    results = []
    
    for race_id in df.index.get_level_values('race_id').unique():
        race_df = df.loc[race_id]
        
        # 3着以内候補の選定
        candidates = race_df[
            ((race_df['score_0'] >= thresholds['score_0']) & (race_df['score_ranking_0'] <= thresholds['score_ranking_0'])) |
            ((race_df['score_1'] >= thresholds['score_1']) & (race_df['score_ranking_1'] <= thresholds['score_ranking_1'])) |
            ((race_df['score_2'] >= thresholds['score_2']) & (race_df['score_ranking_2'] <= thresholds['score_ranking_2']))
        ].index.values
        
        total_bet = 0
        payout = 0
        hit_count = 0
        
        # 3頭BOXの組み合わせ
        for horse1, horse2, horse3 in combinations(candidates, 3):
            combination = f"{horse1:02d}-{horse2:02d}-{horse3:02d}"
            
            # オッズをチェック
            if race_id in odds['sanrenpuku'].index and combination in odds['sanrenpuku'].columns:
                odds_value = odds['sanrenpuku'].loc[race_id, combination]
                if odds_value >= thresholds['min_odds']:
                    total_bet += 100
                    
                    # 的中チェック
                    if race_id in haraimodoshi['sanrenpuku'].index:
                        payout_value = haraimodoshi['sanrenpuku'].loc[race_id, combination]
                        if payout_value > 0:
                            payout = payout_value
                            hit_count = 1
                            break
        
        if total_bet > 0:
            results.append({
                'race_id': race_id,
                'total_bet': total_bet,
                'payout': payout,
                'hit_count': hit_count,
                'purchase_count': len(list(combinations(candidates, 3)))
            })
    
    return pd.DataFrame(results)
    
# 元の三連単シミュレーション関数をここに再定義（インポートできない場合）
def simulate_sanrentan_formation(df, odds, haraimodoshi, thresholds):
    """
    三連単フォーメーション購入シミュレーション（再定義版）
    """
    results = []
    
    for race_id in df.index.get_level_values('race_id').unique():
        race_df = df.loc[race_id]
        
        # 1着候補の選定
        first_candidates = race_df[
            (race_df['score_0'] >= thresholds['score_0']) & 
            (race_df['score_ranking_0'] <= thresholds['score_ranking_0'])
        ].index.values
        
        # 2着候補の選定
        second_candidates = race_df[
            (race_df['score_1'] >= thresholds['score_1']) & 
            (race_df['score_ranking_1'] <= thresholds['score_ranking_1'])
        ].index.values
        
        # 3着候補の選定
        third_candidates = race_df[
            (race_df['score_2'] >= thresholds['score_2']) & 
            (race_df['score_ranking_2'] <= thresholds['score_ranking_2'])
        ].index.values
        
        # フォーメーション購入対象の組み合わせ生成
        purchases = []
        total_bet = 0
        
        for first, second, third in product(first_candidates, second_candidates, third_candidates):
            if first != second and second != third and first != third:
                # 組み合わせ文字列を作成
                combination = f"{first:02d}-{second:02d}-{third:02d}"
                
                # オッズをチェック
                if race_id in odds['sanrentan'].index:
                    if combination in odds['sanrentan'].columns:
                        odds_value = odds['sanrentan'].loc[race_id, combination]
                        if odds_value >= thresholds['min_odds']:
                            purchases.append(combination)
                            total_bet += 100  # 100円購入と仮定
        
        # 的中チェック
        payout = 0
        hit_count = 0
        if race_id in haraimodoshi['sanrentan'].index and purchases:
            for combination in purchases:
                if combination in haraimodoshi['sanrentan'].columns:
                    payout_value = haraimodoshi['sanrentan'].loc[race_id, combination]
                    if payout_value > 0:
                        payout = payout_value
                        hit_count = 1
                        break
        
        if total_bet > 0:
            results.append({
                'race_id': race_id,
                'total_bet': total_bet,
                'payout': payout,
                'hit_count': hit_count,
                'purchase_count': len(purchases)
            })
    
    return pd.DataFrame(results)


# 効率化された馬券種別閾値組み合わせ生成関数
def create_optimized_threshold_combinations_by_betting_type():
    """
    馬券種ごに必要な閾値のみを生成して効率化
    """
    score_candidates = [0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4]
    ranking_candidates = [1, 18]
    min_odds_candidates = [1.5, 2, 3, 4, 5.0, 7.5, 10.0, 15.0, 20.0, 30.0, 40, 50]
    
    betting_type_configs = {
        'tansho': {
            'score_vars': ['score_0'],  # 1着のみ
            'ranking_vars': ['score_ranking_0']
        },
        'fukusho': {
            'score_vars': ['score_2'],  # 3着以内（最低スコア）
            'ranking_vars': ['score_ranking_2']
        },
        'umaren': {
            'score_vars': ['score_2'],  # 2着以内（最低スコア）
            'ranking_vars': ['score_ranking_2']
        },
        'wide': {
            'score_vars': ['score_2'],  # 3着以内（最低スコア）
            'ranking_vars': ['score_ranking_2']
        },
        'umatan': {
            'score_vars': ['score_0', 'score_1'],  # 1着、2着
            'ranking_vars': ['score_ranking_0', 'score_ranking_1']
        },
        'sanrenpuku': {
            'score_vars': ['score_2'],  # 3着以内（最低スコア）
            'ranking_vars': ['score_ranking_2']
        },
        'sanrentan': {
            'score_vars': ['score_0', 'score_1', 'score_2'],  # 1着、2着、3着
            'ranking_vars': ['score_ranking_0', 'score_ranking_1', 'score_ranking_2']
        }
    }
    
    all_combinations = {}
    
    for bet_type, config in betting_type_configs.items():
        combinations = []
        
        # 必要なスコア変数の組み合わせを生成
        score_vars = config['score_vars']
        ranking_vars = config['ranking_vars']
        
        if len(score_vars) == 1:
            # 単一スコア（単勝、複勝、馬連、ワイド、3連複）
            for score in score_candidates:
                for ranking in ranking_candidates:
                    for min_odds in min_odds_candidates:
                        thresholds = {
                            'min_odds': min_odds
                        }
                        # 全てのスコア変数を設定（使用しないものはダミー値）
                        thresholds['score_0'] = score if 'score_0' in score_vars else 0.1
                        thresholds['score_1'] = score if 'score_1' in score_vars else 0.1
                        thresholds['score_2'] = score if 'score_2' in score_vars else 0.1
                        thresholds['score_ranking_0'] = ranking if 'score_ranking_0' in ranking_vars else 18
                        thresholds['score_ranking_1'] = ranking if 'score_ranking_1' in ranking_vars else 18
                        thresholds['score_ranking_2'] = ranking if 'score_ranking_2' in ranking_vars else 18
                        combinations.append(thresholds)
                        
        elif len(score_vars) == 2:
            # 2つのスコア（馬単）
            for score_0 in score_candidates:
                for score_1 in score_candidates:
                    for ranking_0 in ranking_candidates:
                        for ranking_1 in ranking_candidates:
                            for min_odds in min_odds_candidates:
                                thresholds = {
                                    'score_0': score_0,
                                    'score_1': score_1,
                                    'score_2': 0.1,  # ダミー値
                                    'score_ranking_0': ranking_0,
                                    'score_ranking_1': ranking_1,
                                    'score_ranking_2': 18,  # ダミー値
                                    'min_odds': min_odds
                                }
                                combinations.append(thresholds)
                                
        else:
            # 3つのスコア（3連単）
            for score_0 in score_candidates:
                for score_1 in score_candidates:
                    for score_2 in score_candidates:
                        for ranking_0 in ranking_candidates:
                            for ranking_1 in ranking_candidates:
                                for ranking_2 in ranking_candidates:
                                    for min_odds in min_odds_candidates:
                                        thresholds = {
                                            'score_0': score_0,
                                            'score_1': score_1,
                                            'score_2': score_2,
                                            'score_ranking_0': ranking_0,
                                            'score_ranking_1': ranking_1,
                                            'score_ranking_2': ranking_2,
                                            'min_odds': min_odds
                                        }
                                        combinations.append(thresholds)
        
        all_combinations[bet_type] = combinations
        print(f"{bet_type}: {len(combinations):,} combinations")
    
    return all_combinations

def evaluate_optimized_betting_types_worker(args):
    """
    最適化された馬券種評価のワーカー関数
    """
    bet_type, thresholds, df, odds, haraimodoshi = args
    
    simulators = {
        'tansho': simulate_tansho_betting,
        'fukusho': simulate_fukusho_betting,
        'umaren': simulate_umaren_betting,
        'wide': simulate_wide_betting,
        'umatan': simulate_umatan_betting,
        'sanrenpuku': simulate_sanrenpuku_betting,
        'sanrentan': simulate_sanrentan_formation,
    }
    
    try:
        simulator = simulators[bet_type]
        sim_results = simulator(df, odds, haraimodoshi, thresholds)
        
        if len(sim_results) == 0:
            result = {
                'total_bet': 0,
                'total_payout': 0,
                'hit_count': 0,
                'recovery_rate': 0,
                'max_payout': 0,
                'evaluation_score': -np.inf,
                'race_count': 0
            }
        else:
            total_bet = sim_results['total_bet'].sum()
            total_payout = sim_results['payout'].sum()
            hit_count = sim_results['hit_count'].sum()
            max_payout = sim_results['payout'].max()
            
            # 回収率計算
            recovery_rate = total_payout / total_bet if total_bet > 0 else 0
            
            # 評価値計算の修正
            if hit_count > 0 and total_bet > 0:
                # 的中馬券の払戻金額を取得（0より大きいもののみ）
                winning_payouts = sim_results[sim_results['payout'] > 0]['payout'].values
                
                if len(winning_payouts) > 0:
                    # 上位10%を除外する計算
                    top_10_percent_count = max(1, int(len(winning_payouts) * 0.01))
                    
                    # 払戻金額を降順にソート
                    sorted_payouts = np.sort(winning_payouts)[::-1]
                    
                    # 上位10%を除外
                    excluded_payouts = sorted_payouts[:top_10_percent_count]
                    adjusted_total_payout = total_payout - np.sum(excluded_payouts)
                    
                    # 調整済み回収率を計算
                    adjusted_recovery_rate = adjusted_total_payout / total_bet
                    
                    # スコア = log(調整済み回収率) * 的中回数
                    if adjusted_recovery_rate > 0:
                        evaluation_score = np.log(adjusted_recovery_rate) * hit_count
                    else:
                        evaluation_score = -np.inf  # 回収率が0以下の場合は最低スコア
                else:
                    evaluation_score = -np.inf
            else:
                evaluation_score = -np.inf
            
            result = {
                'total_bet': total_bet,
                'total_payout': total_payout,
                'hit_count': hit_count,
                'recovery_rate': recovery_rate,
                'max_payout': max_payout,
                'evaluation_score': evaluation_score,
                'race_count': len(sim_results)
            }
        
        # 閾値情報を結果に追加
        result.update(thresholds)
        return bet_type, result
        
    except Exception as e:
        print(f"Error in {bet_type} simulation: {e}")
        # エラーの場合はダミーの結果を返す
        empty_result = {
            'total_bet': 0,
            'total_payout': 0,
            'hit_count': 0,
            'recovery_rate': 0,
            'max_payout': 0,
            'evaluation_score': -np.inf,
            'race_count': 0
        }
        empty_result.update(thresholds)
        return bet_type, empty_result

def grid_search_optimized_betting_types(df, odds, haraimodoshi, n_workers=None):
    """
    最適化された全馬券種の並列グリッドサーチ
    """
    if n_workers is None:
        n_workers = min(mp.cpu_count(), 8)
    
    print(f"使用プロセス数: {n_workers}")
    
    # 馬券種別の閾値組み合わせを生成
    all_combinations = create_optimized_threshold_combinations_by_betting_type()
    
    # 総組み合わせ数を計算
    total_combinations = sum(len(combinations) for combinations in all_combinations.values())
    print(f"総組み合わせ数: {total_combinations:,}")
    
    # 引数リストを作成
    args_list = []
    for bet_type, combinations in all_combinations.items():
        for thresholds in combinations:
            args_list.append((bet_type, thresholds, df, odds, haraimodoshi))
    
    # 各馬券種の最良結果を追跡
    bet_types = list(all_combinations.keys())
    best_scores = {bet_type: -np.inf for bet_type in bet_types}  # -1から-np.infに変更
    best_thresholds = {bet_type: None for bet_type in bet_types}
    best_results = {bet_type: None for bet_type in bet_types}
    all_results = {bet_type: [] for bet_type in bet_types}
    
    start_time = time.time()
    
    # 並列処理で実行
    with ProcessPoolExecutor(max_workers=n_workers) as executor:
        future_to_args = {executor.submit(evaluate_optimized_betting_types_worker, args): args for args in args_list}
        
        with tqdm(total=total_combinations, desc='Optimized Grid Search', 
                 bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}]') as pbar:
            
            completed_count = 0
            for future in as_completed(future_to_args):
                try:
                    bet_type, result = future.result()
                    all_results[bet_type].append(result)
                    
                    # 最良の結果を更新
                    if result['evaluation_score'] > best_scores[bet_type]:
                        best_scores[bet_type] = result['evaluation_score']
                        best_thresholds[bet_type] = {k: result[k] for k in ['score_0', 'score_1', 'score_2', 
                                                                           'score_ranking_0', 'score_ranking_1', 
                                                                           'score_ranking_2', 'min_odds']}
                        best_results[bet_type] = result.copy()
                    
                    completed_count += 1
                    pbar.update(1)
                    
                    # 10%ごとに中間結果を表示
                    if completed_count % (total_combinations // 10) == 0:
                        elapsed_time = time.time() - start_time
                        progress_pct = (completed_count / total_combinations) * 100
                        pbar.set_postfix({
                            'Progress': f'{progress_pct:.1f}%',
                            'Elapsed': f'{elapsed_time:.1f}s'
                        })
                        
                except Exception as e:
                    print(f"タスクでエラーが発生: {e}")
                    pbar.update(1)
    
    elapsed_time = time.time() - start_time
    print(f"\n並列処理完了時間: {elapsed_time:.2f}秒")
    print(f"処理効率: {total_combinations / elapsed_time:.1f} combinations/sec")
    
    # DataFrameに変換
    all_results_df = {}
    for bet_type in bet_types:
        all_results_df[bet_type] = pd.DataFrame(all_results[bet_type])
    
    return best_thresholds, best_results, all_results_df

In [ ]:
# 実行例
print("全馬券種購入シミュレーションを開始...")
print(f"利用可能CPU数: {mp.cpu_count()}")

# 全馬券種のグリッドサーチを実行
best_thresholds_all, best_results_all, all_results_all = grid_search_optimized_betting_types(
    df, odds, haraimodoshi, n_workers=16
)

In [ ]:
# 結果の表示

# 1. 各馬券種の最良結果サマリーを表示
print("\n" + "="*80)
print("各馬券種の最良結果サマリー")
print("="*80)

summary_data = []
for bet_type in ['tansho', 'fukusho', 'umaren', 'wide', 'umatan', 'sanrenpuku', 'sanrentan']:
    if bet_type in best_results_all and best_results_all[bet_type] is not None:
        result = best_results_all[bet_type]
        summary_data.append({
            '馬券種': bet_type,
            '総投資額': f"{result['total_bet']:,}円",
            '総払戻': f"{result['total_payout']:,}円",
            '回収率': f"{result['recovery_rate']:.3f}",
            '的中回数': result['hit_count'],
            'レース数': result['race_count'],
            '最大払戻': f"{result['max_payout']:,}円",
            '評価スコア': f"{result['evaluation_score']:.6f}"
        })

summary_df = pd.DataFrame(summary_data)
print(summary_df.to_string(index=False))

# 2. 回収率上位の馬券種をグラフ表示
plt.figure(figsize=(12, 8))

# 回収率のグラフ
plt.subplot(2, 2, 1)
recovery_rates = [best_results_all[bet_type]['recovery_rate'] for bet_type in ['tansho', 'fukusho', 'umaren', 'wide', 'umatan', 'sanrenpuku', 'sanrentan'] if bet_type in best_results_all and best_results_all[bet_type] is not None]
bet_types_available = [bet_type for bet_type in ['tansho', 'fukusho', 'umaren', 'wide', 'umatan', 'sanrenpuku', 'sanrentan'] if bet_type in best_results_all and best_results_all[bet_type] is not None]

bars = plt.bar(bet_types_available, recovery_rates, color='skyblue', alpha=0.7)
plt.axhline(y=1.0, color='red', linestyle='--', alpha=0.7, label='損益分岐点')
plt.ylabel('回収率')
plt.title('各馬券種の回収率')
plt.xticks(rotation=45)
plt.legend()
plt.grid(True, alpha=0.3)

# 値をバーの上に表示
for bar, rate in zip(bars, recovery_rates):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
             f'{rate:.3f}', ha='center', va='bottom', fontsize=9)

# 的中回数のグラフ
plt.subplot(2, 2, 2)
hit_counts = [best_results_all[bet_type]['hit_count'] for bet_type in bet_types_available]
bars = plt.bar(bet_types_available, hit_counts, color='lightgreen', alpha=0.7)
plt.ylabel('的中回数')
plt.title('各馬券種の的中回数')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

for bar, count in zip(bars, hit_counts):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
             f'{count}', ha='center', va='bottom', fontsize=9)

# 評価スコアのグラフ
plt.subplot(2, 2, 3)
eval_scores = [best_results_all[bet_type]['evaluation_score'] for bet_type in bet_types_available]
bars = plt.bar(bet_types_available, eval_scores, color='orange', alpha=0.7)
plt.ylabel('評価スコア')
plt.title('各馬券種の評価スコア')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

for bar, score in zip(bars, eval_scores):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.001, 
             f'{score:.4f}', ha='center', va='bottom', fontsize=8)

# 投資額vs払戻額の散布図
plt.subplot(2, 2, 4)
total_bets = [best_results_all[bet_type]['total_bet'] for bet_type in bet_types_available]
total_payouts = [best_results_all[bet_type]['total_payout'] for bet_type in bet_types_available]

plt.scatter(total_bets, total_payouts, s=100, alpha=0.7, c=recovery_rates, cmap='RdYlGn', vmin=0.5, vmax=1.5)
plt.colorbar(label='回収率')

# 損益分岐線を引く
max_val = max(max(total_bets), max(total_payouts))
plt.plot([0, max_val], [0, max_val], 'r--', alpha=0.7, label='損益分岐線')

plt.xlabel('総投資額 (円)')
plt.ylabel('総払戻額 (円)')
plt.title('投資額 vs 払戻額')
plt.legend()
plt.grid(True, alpha=0.3)

# 各点にラベルを追加
for i, bet_type in enumerate(bet_types_available):
    plt.annotate(bet_type, (total_bets[i], total_payouts[i]), 
                xytext=(5, 5), textcoords='offset points', fontsize=8)

plt.tight_layout()
plt.show()

# 3. 各馬券種の最適閾値を表示
print("\n" + "="*80)
print("各馬券種の最適閾値")
print("="*80)

threshold_data = []
for bet_type in bet_types_available:
    if bet_type in best_thresholds_all and best_thresholds_all[bet_type] is not None:
        thresholds = best_thresholds_all[bet_type]
        threshold_data.append({
            '馬券種': bet_type,
            'score_0': f"{thresholds['score_0']:.2f}",
            'score_1': f"{thresholds['score_1']:.2f}",
            'score_2': f"{thresholds['score_2']:.2f}",
            'ranking_0': thresholds['score_ranking_0'],
            'ranking_1': thresholds['score_ranking_1'],
            'ranking_2': thresholds['score_ranking_2'],
            'min_odds': f"{thresholds['min_odds']:.1f}"
        })

threshold_df = pd.DataFrame(threshold_data)
print(threshold_df.to_string(index=False))

# 4. 回収率上位3馬券種の詳細分析
print("\n" + "="*80)
print("回収率上位3馬券種の詳細分析")
print("="*80)

# 回収率でソート
recovery_ranking = sorted([(bet_type, best_results_all[bet_type]['recovery_rate']) 
                          for bet_type in bet_types_available], 
                         key=lambda x: x[1], reverse=True)

top_3_bet_types = [x[0] for x in recovery_ranking[:3]]

for i, bet_type in enumerate(top_3_bet_types, 1):
    result = best_results_all[bet_type]
    print(f"\n{i}位: {bet_type}")
    print("-" * 40)
    print(f"回収率: {result['recovery_rate']:.3f}")
    print(f"総投資額: {result['total_bet']:,}円")
    print(f"総払戻額: {result['total_payout']:,}円")
    print(f"利益: {result['total_payout'] - result['total_bet']:,}円")
    print(f"的中回数: {result['hit_count']}回")
    print(f"参加レース数: {result['race_count']}レース")
    print(f"的中率: {result['hit_count']/result['race_count']*100:.1f}%")
    print(f"最大払戻: {result['max_payout']:,}円")

# 5. 全体サマリー
print("\n" + "="*80)
print("全体サマリー")
print("="*80)

total_investment = sum(best_results_all[bet_type]['total_bet'] for bet_type in bet_types_available)
total_return = sum(best_results_all[bet_type]['total_payout'] for bet_type in bet_types_available)
total_profit = total_return - total_investment
overall_recovery_rate = total_return / total_investment if total_investment > 0 else 0

print(f"総投資額: {total_investment:,}円")
print(f"総払戻額: {total_return:,}円")
print(f"総利益: {total_profit:,}円")
print(f"全体回収率: {overall_recovery_rate:.3f}")

if overall_recovery_rate > 1.0:
    print("✅ 全体として利益が出ています！")
else:
    print("❌ 全体として損失が出ています。")

# プラス収支の馬券種数
profitable_count = sum(1 for bet_type in bet_types_available if best_results_all[bet_type]['recovery_rate'] > 1.0)
print(f"プラス収支の馬券種数: {profitable_count}/{len(bet_types_available)}")